<a href="https://colab.research.google.com/github/Djadih/DataScience_Project/blob/main/midi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [234]:
from google.colab import files
uploaded = files.upload()

Saving mz_570_3.mid to mz_570_3.mid


In [189]:
import string
import numpy as np
from music21 import converter, instrument, note, chord, stream, tempo, meter

In [190]:
midi = converter.parse('mz_331_1.mid')

In [191]:
max = 10
j = 0
ins = instrument.partitionByInstrument(midi)
notes_to_parse = ins.parts[0].recurse() 
for i in notes_to_parse:
  print(i, i.offset)
  j+=1
  if j ==max:
    break

Piano 0.0
<music21.tempo.MetronomeMark Quarter=63.51> 0.0
A major 0.0
A major 0.0
<music21.meter.TimeSignature 6/8> 0.0
<music21.meter.TimeSignature 6/8> 0.0
<music21.note.Note C#> 0.0
<music21.note.Rest rest> 0.0
<music21.note.Note A> 0.0
<music21.note.Note E> 0.0


In [192]:
def get_notes(filename):
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []
    temp = 0
    temp_string = ''
    temp_found = False
    st = ''
    midi = converter.parse(filename)
    print("Parsing %s" % filename)
    notes_to_parse = None

    try: # file has instrument parts
      ins = instrument.partitionByInstrument(midi)
      notes_to_parse = ins.parts[0].recurse() 
    except: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
      if isinstance(element, tempo.MetronomeMark) and not temp_found:
        temp = element.getQuarterBPM()
        temp_string = element.text
        temp_found = True
      elif isinstance(element, meter.TimeSignature):
        st = element.ratioString

      elif isinstance(element, note.Note):
        notes.append(([element.pitch.midi], element.offset))
      elif isinstance(element, chord.Chord):
        notes.append(([n.midi for n in element.pitches], element.offset))

    return notes, temp, temp_string, st

def get_dict(filename):
    notes, temp, temp_string, st = get_notes(filename)
    music = {}
    music['tempo_num'] = temp
    music['tempo_string'] = temp_string
    music['time_signature'] = st
    return music, notes

In [235]:
song, notes = get_dict('mz_570_3.mid')

Parsing mz_570_3.mid


In [236]:
song

{'tempo_num': 156.21, 'tempo_string': None, 'time_signature': '4/4'}

In [237]:
for i in notes[:30]:
  print(i)

([70], 3.5)
([74], 4.0)
([58], 4.0)
([65], 4.5)
([75], 5.0)
([60], 5.0)
([65], 5.5)
([76], 6.0)
([62], 6.0)
([65], 6.5)
([77], 7.0)
([58], 7.0)
([74], 7.5)
([65], 7.5)
([74], 7.75)
([72], 8.0)
([57], 8.0)
([71], 8.5)
([65], 8.5)
([72], 9.0)
([58], 9.0)
([74], 9.5)
([65], 9.5)
([77], 10.0)
([60], 10.0)
([65], 10.5)
([75], 11.0)
([63], 11.0)
([69], 11.5)
([65], 11.5)


In [225]:
def notes_to_vec(notes):

  length = int(notes[-1][1]*4)
  vect = np.zeros((length *4 , 88))  
  for chord, offset in notes:
    for note in chord:
      vect[int(offset*4)][note-21] = 1
  return vect

In [238]:
test = notes_to_vec(notes[:500])

In [239]:
test[108].nonzero()

(array([42, 54]),)

In [240]:
def vec_to_notes(vect):
  notes = []
  for idx in range(vect.shape[0]):
    current_note = vect[idx]
    if not np.all(current_note == 0):
      chord = np.array(current_note.nonzero()[0]+21).tolist()
      notes.append((chord, idx/4 ))
  return notes    

In [241]:
test_notes = vec_to_notes(test)

In [242]:
test_notes[:20]

[([70], 3.5),
 ([58, 74], 4.0),
 ([65], 4.5),
 ([60, 75], 5.0),
 ([65], 5.5),
 ([62, 76], 6.0),
 ([65], 6.5),
 ([58, 77], 7.0),
 ([65, 74], 7.5),
 ([74], 7.75),
 ([57, 72], 8.0),
 ([65, 71], 8.5),
 ([58, 72], 9.0),
 ([65, 74], 9.5),
 ([60, 77], 10.0),
 ([65], 10.5),
 ([63, 75], 11.0),
 ([65, 69], 11.5),
 ([62, 70], 12.0),
 ([65, 70], 12.5)]

In [243]:
def create_midi(prediction_dict, notes):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    output_notes = []
    temp_num = prediction_dict['tempo_num']
    temp_string = prediction_dict['tempo_string']
    st = prediction_dict['time_signature']

    mm1 = tempo.MetronomeMark(temp_string, temp_num)
    ts = meter.TimeSignature(st)
    output_notes.append(mm1)
    output_notes.append(ts)

    # create note and chord objects based on the values generated by the model
    for pattern, offset in notes:
        # pattern is a chord
          chord_list = []
          for current_note in pattern:
              new_note = note.Note(current_note)
              new_note.storedInstrument = instrument.Piano()
              chord_list.append(new_note)
          new_chord = chord.Chord(chord_list)
          new_chord.offset = offset
          output_notes.append(new_chord)


        # increase offset each iteration so that not  es do not stack

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output_tempo.mid')

In [244]:
create_midi(song, test_notes)